In [ ]:
import numpy as np
import h5py
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
import string

import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-78,central_longitude=-60)#260-360)
axex = [294.5,300,-69.5,-65.5]
#axex2 = [295,300,-75.5,-75]
#axex3 = [277.2,278.2,-79.2,-78.2]
#axex4 = [290,298,-78.4,-78.2]

cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.rain'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

x0,x1,y0,y1 = 1500,3000,3900,5000

In [ ]:
vmax = 10
vmin = -10
linthresh = .3
linscale = .25


fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#print(fracpos)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,.9,nneg))
colors2 = plt.cm.inferno(np.linspace(0.1, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)


In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
    mask = xr.where(ds.mask==1,2,ds.mask)
    ds = add_lonlat(ds)
    
    ax.set_facecolor(".7")
    
    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    ax.set_extent(axex,crs=ccrs.PlateCarree())

In [ ]:
def add_zoom(ax,axexs):
    for axx in axexs:
        axins = inset_axes(ax,width="0%",height="0%",loc='lower right',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))    
        axins.set_extent(axx,crs=ccrs.PlateCarree())
        ax.indicate_inset_zoom(axins, edgecolor="k",lw=1)
        
    ax.text(.19,.74,'WB',transform=ax.transAxes)
    ax.text(.16,.05,'RG',transform=ax.transAxes)    
    ax.text(.35,.4,'KI',transform=ax.transAxes)    


In [ ]:
def plotmelt(ax,lon,lat,melt,title):
    makebackground(ax,axex)
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,melt,cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),shading='nearest',transform=ccrs.PlateCarree())

    ax.set_title(title,loc='left',fontsize=10)
    
    #add_zoom(ax,[axex2,axex3,axex4])
    
    return im

In [ ]:
#Read observations Adusumilli

fname = '../../../data/adusumilli/bb0448974g_3_1.h5'
hf = h5py.File(fname, 'r')
xa = np.array(hf['/x'])[:,0]
ya = np.array(hf['/y'])[:,0]
melta = np.array(hf['/w_b'])


data_vars = {'melt':(['y','x'],melta)}
coords = {'x': (['x'], xa),'y': (['y'], ya)}

dsa = xr.Dataset(data_vars=data_vars,coords=coords)
dsa = dsa.isel(x=slice(500,2000),y=slice(6000,8000))
dsa = add_lonlat(dsa)

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.01,right=.99,top=.95,bottom=.15,wspace=.05,hspace=.1)

fig = plt.figure(figsize=(7,5.5))

#Geometry

ax = fig.add_subplot(131,projection=proj)
makebackground(ax,axex)
ax.set_extent(axex,crs=ccrs.PlateCarree())
ax.set_title('a) Geometry \n   (Morlighem ea. 2020 )',loc='left',fontsize=10)

ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==3,ds.thickness,np.nan),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=0,vmax=1500)

#add_zoom(ax,[axex2,axex3,axex4])

#Obs
ax = fig.add_subplot(132,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(dsa.lon,dsa.lat,dsa.melt,shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())
ax.set_title('b) Remote sensing \n   (Adusumilli ea. 2020)',loc='left',fontsize=10)

#add_zoom(ax,[axex2,axex3,axex4])

#cax = inset_axes(ax,width="5%",height="100%",loc='lower left',bbox_to_anchor=(1.05,0,1,1),bbox_transform=ax.transAxes,borderpad=0)
#cb = plt.colorbar(im2,cax=cax)
#cb.set_label('Ice shelf thickness [m]')

#LADDIE
ax = fig.add_subplot(133,projection=proj)

ds = xr.open_dataset(f'../../results/LCIS_1.0_linear_S134.8_T1-1.5_050.nc')
plotmelt(ax,ds.lon,ds.lat,np.where(ds.tmask,ds.melt,np.nan),'c) LADDIE model \n   (Lambert ea., in prep)')

ax = fig.add_subplot(111)
ax.set_visible(False)
cax = inset_axes(ax,width="33%",height="5%",loc='lower left',bbox_to_anchor=(0,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
cb = plt.colorbar(im2,cax=cax,orientation='horizontal')
cb.set_label('Ice shelf thickness [m]')

cax = inset_axes(ax,width="65%",height="5%",loc='lower left',bbox_to_anchor=(.35,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cb = plt.colorbar(im,cax=cax,orientation='horizontal',extend='both')
cb.set_ticks(ticks)
cb.set_ticklabels(ticks)
cb.set_label('Basal freezing / melt rate [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_LC_melt.png',dpi=450)